<a href="https://colab.research.google.com/github/SattuSupCodes/SentimentAnalysis_TheoreticalLearning/blob/main/ECAPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ejlok1/cremad")

print("Path to dataset files:", path)

100%|██████████| 451M/451M [00:03<00:00, 151MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ejlok1/cremad/versions/1


# Pin Your Install Versions
when dealing with tools that are not compatible with latest upgrades, the smartest choice (coming from not the brightest star rn) is to pin your versions.

In [ ]:
!pip install --no-cache-dir \
  "torch==2.0.1+cpu" \
  "torchaudio==2.0.2+cpu" \
  --index-url https://download.pytorch.org/whl/cpu

!pip install "speechbrain==0.5.16" soundfile

Looking in indexes: https://download.pytorch.org/whl/cpu
ERROR: Could not find a version that satisfies the requirement torch==2.0.1+cpu (from versions: 2.2.0+cpu, 2.2.1+cpu, 2.2.2+cpu, 2.3.0+cpu, 2.3.1+cpu, 2.4.0+cpu, 2.4.1+cpu, 2.5.0+cpu, 2.5.1+cpu, 2.6.0+cpu, 2.7.0+cpu, 2.7.1+cpu, 2.8.0+cpu, 2.9.0+cpu, 2.9.1+cpu)
ERROR: No matching distribution found for torch==2.0.1+cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 37.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import librosa
import os
import torchaudio
from speechbrain.lobes.models.ECAPA_TDNN import ECAPA_TDNN
from speechbrain.pretrained import EncoderClassifier
import torch
from sklearn.cluster import AgglomerativeClustering
import soundfile as sf



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # imagine not having cuda (heartbreak emoji)

ecapa = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    run_opts={"device": device}
)

ecapa.eval()
print("ECAPA loaded (frozen embedding extractor)")


hyperparams.yaml: 0.00B [00:00, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt: 0.00B [00:00, ?B/s]

ECAPA loaded (frozen embedding extractor)


We downloaded our PRETRAINED ecapa

In [ ]:
# audio_path = os.path.join(
#     path, "AudioWAV", "1001_IEO_SAD_HI.wav"
# )

Lets be kind to our technology. Diarisation is NOT per dataset, its pr audio file.


In [ ]:

wav, sr = sf.read("sample4.wav")

wav = torch.tensor(wav, dtype=torch.float32)

if wav.ndim == 2:   # stereo → mono meaning combining two channels in one because our ecapa can only process mono audios
    wav = wav.mean(dim=1)

print(wav.shape, sr)



torch.Size([4449280]) 44100


In [ ]:
frame_sec = 0.025
frame_len = int(frame_sec * sr)
hop_len = frame_len // 2


In [ ]:
energies = []

for i in range(0, len(wav) - frame_len, hop_len):
    frame = wav[i:i+frame_len]
    energy = np.sum(frame.numpy()**2)
    energies.append(energy)

energies = np.array(energies)

In [ ]:
energy_thresh = 0.03 * np.max(energies)


In [ ]:
speech_mask = energies > energy_thresh


In [ ]:
segments = []
start = None

for i, is_speech in enumerate(speech_mask):
    t = i * hop_len / sr
    if is_speech and start is None:
        start = t
    elif not is_speech and start is not None:
        end = t
        if end - start > 0.5:  # min speech length
            segments.append((start, end))
        start = None

# handle last segment
if start is not None:
    segments.append((start, len(wav)/sr))


In [ ]:
window_sec = 2.5
hop_sec = 1.5

window_samples = int(window_sec * sr)
hop_samples = int(hop_sec * sr)

segments = []

for start in range(0, len(wav) - window_samples, hop_samples):
    segment = wav[start:start + window_samples]
    segments.append(segment)

print(f"Segments created: {len(segments)}")


Segments created: 66


In [ ]:
window_sec = 1.0
hop_sec = 0.5

win_len = int(window_sec * sr)
hop_len = int(hop_sec * sr)

embeddings = []

for speech in segments:   # speech is already a waveform tensor

    if len(speech) < win_len:
        continue

    for i in range(0, len(speech) - win_len + 1, hop_len):
        chunk = speech[i:i+win_len]

        with torch.no_grad():
            emb = ecapa.encode_batch(chunk.unsqueeze(0))

        emb = emb.squeeze(0)
        embeddings.append(emb.cpu().numpy())

if len(embeddings) == 0:
    raise RuntimeError("No embeddings extracted — VAD/windowing mismatch")

embeddings = np.vstack(embeddings)
print("Embeddings shape:", embeddings.shape)



# o/p (number of segments, embedding dimensions)-> speaker info in vector form

NameError: name 'sr' is not defined

In [ ]:
print("Number of VAD segments:", len(segments))
print("First 10 VAD segments (sec):", segments[:10])

durations = [(end - start) for start, end in segments]
print("First 10 segment durations:", durations[:10])
print("Max segment duration:", max(durations) if durations else 0)


Number of VAD segments: 66
First 10 VAD segments (sec): [tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.1231, -0.1335, -0.1505]), tensor([-1.4252e-02, -1.6617e-02, -1.5289e-02,  ..., -3.0518e-05,
        -4.5776e-05, -6.1035e-05]), tensor([0.0175, 0.0242, 0.0286,  ..., 0.0470, 0.0521, 0.0570]), tensor([-0.0003, -0.0003, -0.0003,  ..., -0.0755, -0.0636, -0.0527]), tensor([ 0.0000e+00,  1.5259e-05,  0.0000e+00,  ..., -1.5729e-01,
        -1.4148e-01, -1.2738e-01]), tensor([0.0126, 0.0156, 0.0158,  ..., 0.0636, 0.0610, 0.0578]), tensor([0.0074, 0.0104, 0.0125,  ..., 0.0248, 0.0208, 0.0196]), tensor([ 0.0321,  0.0299,  0.0281,  ..., -0.0041, -0.0066, -0.0090]), tensor([0.0879, 0.0879, 0.0882,  ..., 0.0029, 0.0028, 0.0026]), tensor([0.0007, 0.0009, 0.0011,  ..., 0.0789, 0.0861, 0.0930])]


ValueError: too many values to unpack (expected 2)

In [ ]:
print("Sample rate:", sr)


Sample rate: 44100


In [ ]:
if sr != 16000:
    wav = torchaudio.functional.resample(wav, sr, 16000)
    sr = 16000


In [ ]:

print("Sample rate:", sr)

Sample rate: 16000


its like idek you anymore, my model

In [ ]:
print(type(emb), emb.shape)


<class 'torch.Tensor'> torch.Size([1, 192])


In [ ]:

clusterer = AgglomerativeClustering(
    n_clusters=None,
    metric="cosine",
    linkage="average",
    distance_threshold=0.7
)

labels = clusterer.fit_predict(embeddings)
print("Speakers detected:", len(set(labels)))


Speakers detected: 1


In [ ]:
prev = labels[0]
print(f"0.00s → Speaker {prev}")

for i in range(1, len(labels)):
    if labels[i] != prev:
        t = i * hop_sec
        print(f"{t:.2f}s → Speaker {labels[i]}")
        prev = labels[i]


0.00s → Speaker 0
